<a href="https://colab.research.google.com/github/rygonzalez/ML-Colab-Notebooks/blob/main/Arduino_Face_Recognition_File.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#include <Keypad.h>
#include <Servo.h>
Servo myServo;

int const ServoPin = 11;    // Servo control ouput pin
int const BuzzPin = 10;     // Buzzer control ouput pin
int const Unlock = 70;      // Servo angle to UNLOCK door
int const Lock = 130;       // Servo angle to LOCK door
int const ROW_NUM = 4;      //four rows
int const COLUMN_NUM = 3;   //three columns
int const Length = 7;       // how long the keypad password can be. 6 digits plus 1 null = 7
char PASSWORD[] = "123456"; // correct password
char Input[Length];
byte index = 0, attempts = 0;

char keys[ROW_NUM][COLUMN_NUM] = {
  {'1','2','3'},
  {'4','5','6'},
  {'7','8','9'},
  {'*','0','#'}
};

byte pin_rows[ROW_NUM] = {8, 7, 6, 5}; //connect to the row pinouts of the keypad
byte pin_column[COLUMN_NUM] = {4, 3, 2}; //connect to the column pinouts of the keypad

Keypad keypad = Keypad( makeKeymap(keys), pin_rows, pin_column, ROW_NUM, COLUMN_NUM );
// Define Functions:
int BuzzerWrongFunction(){
    tone(BuzzPin, 1000);
    delay(200);
    tone(BuzzPin, 700);
    delay(200);
    tone(BuzzPin, 400);
    delay(200);
    noTone(BuzzPin);
    delay(50);
}
int BuzzerRightFunction(){
    tone(BuzzPin, 400);
    delay(200);
    tone(BuzzPin, 700);
    delay(200);
    tone(BuzzPin, 1000);
    delay(200);
    noTone(BuzzPin);
    delay(50);
}
int ClearData(){
  while (index != 0){
    Input[index--] = 0;
  }
}
int LockDoor(){
    myServo.write(Lock);
    delay(50);
}
int UnlockDoor(){
  myServo.write(Unlock);
  delay(5000);          // five second delay for time to open door
  myServo.write(Lock);
  delay(50);            // lock door at end
}

void setup(){

  pinMode(BuzzPin, OUTPUT);  // Set BuzzerPin as output
  myServo.attach(ServoPin);  // Assign ServoPin to the servo

  Serial.begin(9600);
  myServo.write(Lock);       // Initial door state is locked
}

void loop(){
  char key = keypad.getKey();  // Read input from keypad
  if (key){
    if (key == '*'){ // If 'START' key on keypad is pressed
      if (Serial.available() > 0) { // Check if message is available in serial com (from Raspberry Pi)
        String data = Serial.readStringUntil('\n');  // Classify incoming message
        if (data == "555"){ // if RPi is sending '555/n'
          UnlockDoor();
          delay(20);
        }
      }
    }
    else if (key == '#'){ // If 'STOP' button on keypad is pressed, clear input string
      ClearData();
    }
    else{ // If any other button on the keypad is pressed, append the input string with that digit
      Input[index] = key;
      index++;
      tone(BuzzPin, 700);
      delay(50);
      noTone(BuzzPin);
      delay(50);
    }
  }
  if (index == Length-1){ // If there are 6 entries in the input string
    if (!strcmp(Input,PASSWORD)){  //If the input string matches the password
      BuzzerRightFunction();  // Success buzzer sound
      UnlockDoor();
      delay(20);
      ClearData();        // Clear input string data
  }
    else{   // If the input string does not match the password
      ClearData();
      BuzzerWrongFunction();
  }
}
delay(20);
}